<a href="https://colab.research.google.com/github/Suwannaket1/bsc_dpdm24/blob/main/HW3_Group2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

นายคณิศร กุดกลาง         รหัสนักศึกษา 653020202-7

นางสาวปีย์รดา ภู่ถนนนอก    รหัสนักศึกษา 653020211-6

นางสาวสุวรรณเกตุ สุมาลี    รหัสนักศึกษา 653020578-2

# Import Data

In [ ]:
import pandas as pd
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/DPDM24'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv(os.path.join(path,'reduced_marketbasket.csv'),encoding='latin-1')

In [ ]:
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/01/2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/01/2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/01/2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/01/2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/01/2010 08:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/09/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/09/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/09/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/09/2011 12:50,4.15,12680.0,France


# Clean Data

In [ ]:
data.isnull().sum()

,0
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0


In [ ]:
data = data.dropna(subset=['CustomerID'])
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/01/2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/01/2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/01/2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/01/2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/01/2010 08:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/09/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/09/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/09/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/09/2011 12:50,4.15,12680.0,France


In [ ]:
remaining_percentage = (len(data) / len(pd.read_csv(os.path.join(path,'reduced_marketbasket.csv'),encoding='latin-1'))) * 100
print(f'{remaining_percentage:.2f}% of data remains after removing null values.')

75.07% of data remains after removing null values.


In [ ]:
data.isnull().sum()

,0
InvoiceNo,0
StockCode,0
Description,0
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,0
Country,0


# 1. วิเคราะห์พฤติกรรมผู้บริโภค (groupby customer id) แต่ละทวีป

In [ ]:
!pip install pycountry_convert

In [ ]:
!pip install apyori

In [ ]:
import pandas as pd
import pycountry_convert as pc
from apyori import apriori

def country_to_continent(country_name):
    try:
        country_map = {
            'EIRE': 'IE',
            'Channel Islands': 'JE',
            'United Kingdom': 'GB'
        }

        country_code = country_map.get(country_name)
        if not country_code:
            country_code = pc.country_name_to_country_alpha2(country_name)

        continent_code = pc.country_alpha2_to_continent_code(country_code)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)

        return continent_name
    except:
        return 'Unknown'

# เพิ่มคอลัมน์ทวีป
data['Continent'] = data['Country'].apply(country_to_continent)

# จัดกลุ่มตาม CustomerID
customer_transactions = data.groupby('CustomerID')['StockCode'].apply(list)

# สร้าง Association Rules
association_rules = list(apriori(customer_transactions, min_support=0.03))

# กรองกฏที่มีมากกว่า 1 item
complex_rules = [rule for rule in association_rules if len(rule.items) > 1]

# แปลงกฏให้อ่านง่าย
def format_rules(rules):
    formatted = []
    for rule in rules:
        items = list(rule.items)
        support = rule.support
        rule_details = {
            'items': items,
            'support': support
        }
        formatted.append(rule_details)
    return formatted

# จัดเก็บผลลัพธ์
result_rules = format_rules(complex_rules)

print("จำนวนกฏความสัมพันธ์:", len(result_rules))
print("\nตัวอย่างกฏความสัมพันธ์:")
for rule in result_rules[:5]:
    print(rule)

<ipython-input-31-71cebb986b9e>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Continent'] = data['Country'].apply(country_to_continent)


จำนวนกฏความสัมพันธ์: 1354

ตัวอย่างกฏความสัมพันธ์:
{'items': ['20685', '22423'], 'support': 0.030420860018298263}
{'items': ['20685', '85099B'], 'support': 0.03339432753888381}
{'items': ['20726', '20712'], 'support': 0.030649588289112534}
{'items': ['20712', '22386'], 'support': 0.030420860018298263}
{'items': ['20712', '23203'], 'support': 0.030420860018298263}


In [ ]:
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Continent
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/01/2010 08:26,2.55,17850.0,United Kingdom,Europe
1,536365,71053,WHITE METAL LANTERN,6,12/01/2010 08:26,3.39,17850.0,United Kingdom,Europe
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/01/2010 08:26,2.75,17850.0,United Kingdom,Europe
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/01/2010 08:26,3.39,17850.0,United Kingdom,Europe
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/01/2010 08:26,3.39,17850.0,United Kingdom,Europe
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/09/2011 12:50,0.85,12680.0,France,Europe
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/09/2011 12:50,2.10,12680.0,France,Europe
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/09/2011 12:50,4.15,12680.0,France,Europe
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/09/2011 12:50,4.15,12680.0,France,Europe


In [ ]:
# คอลัมน์ Continent ที่เป็น Unknow มี Country อะไรบ้าง
unknown_countries = data[data['Continent'] == 'Unknown']['Country'].unique()
unknown_countries

array(['Unspecified', 'European Community', 'RSA'], dtype=object)

In [ ]:
# อัพเดททวีปเพิ่มเติม
def custom_country_to_continent(country_name):
    special_mapping = {
        'RSA': 'Africa',  # Republic of South Africa
        'European Community': 'Europe',
        'Unspecified': 'Unknown'
    }

    return special_mapping.get(country_name, country_to_continent(country_name))

# อัพเดทการแมปทวีป
data['Continent'] = data['Country'].apply(custom_country_to_continent)

<ipython-input-34-c6b5c2123836>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Continent'] = data['Country'].apply(custom_country_to_continent)


In [ ]:
# คอลัมน์ Continent ที่เป็น Unknow มี Country อะไรบ้าง
unknown_countries = data[data['Continent'] == 'Unknown']['Country'].unique()
unknown_countries

array(['Unspecified'], dtype=object)

In [ ]:
# กรองกฏความสัมพันธ์ตามทวีป
continent_rules = {}

for rule in result_rules:
    items = list(rule['items'])

    # หาทวีปของสินค้า
    item_continents = set()
    for item in items:
        continents = set(data[data['StockCode'] == item]['Continent'].unique())
        item_continents.update(continents)

    # รวบรวมกฏตามทวีป
    for continent in item_continents:
        if continent not in continent_rules:
            continent_rules[continent] = []

        continent_rules[continent].append({
            'items': items,
            'support': rule['support'],
            'descriptions': [
                data[data['StockCode'] == item]['Description'].unique()[0]
                for item in items
            ]
        })

# แสดงสรุปกฏความสัมพันธ์ตามทวีป
for continent, rules in continent_rules.items():
    print(f"\nทวีป: {continent}")
    print(f"จำนวนกฏความสัมพันธ์: {len(rules)}")

    # แสดง 5 กฏแรกของแต่ละทวีป
    for rule in rules[:5]:
        print(f"สินค้า: {rule['items']}")
        print(f"รายละเอียด: {rule['descriptions']}")
        print(f"Support: {rule['support']:.4f}\n")


ทวีป: Oceania
จำนวนกฏความสัมพันธ์: 1259
สินค้า: ['20685', '22423']
รายละเอียด: ['DOORMAT RED RETROSPOT', 'REGENCY CAKESTAND 3 TIER']
Support: 0.0304

สินค้า: ['20685', '85099B']
รายละเอียด: ['DOORMAT RED RETROSPOT', 'JUMBO BAG RED RETROSPOT']
Support: 0.0334

สินค้า: ['20726', '20712']
รายละเอียด: ['LUNCH BAG WOODLAND', 'JUMBO BAG WOODLAND ANIMALS']
Support: 0.0306

สินค้า: ['20712', '22386']
รายละเอียด: ['JUMBO BAG WOODLAND ANIMALS', 'JUMBO BAG PINK POLKADOT']
Support: 0.0304

สินค้า: ['20712', '23203']
รายละเอียด: ['JUMBO BAG WOODLAND ANIMALS', 'JUMBO BAG DOILEY PATTERNS']
Support: 0.0304


ทวีป: Asia
จำนวนกฏความสัมพันธ์: 1298
สินค้า: ['20685', '22423']
รายละเอียด: ['DOORMAT RED RETROSPOT', 'REGENCY CAKESTAND 3 TIER']
Support: 0.0304

สินค้า: ['20685', '85099B']
รายละเอียด: ['DOORMAT RED RETROSPOT', 'JUMBO BAG RED RETROSPOT']
Support: 0.0334

สินค้า: ['20726', '20712']
รายละเอียด: ['LUNCH BAG WOODLAND', 'JUMBO BAG WOODLAND ANIMALS']
Support: 0.0306

สินค้า: ['20712', '22386']
รายละเ

# 2.ลูกค้านิยมซื้อสินค้าใดร่วมกัน (2 Item) ในแต่ละฤดูกาล

In [ ]:
data

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/01/2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/01/2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/01/2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/01/2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/01/2010 08:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/09/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/09/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/09/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/09/2011 12:50,4.15,12680.0,France


## Prepare Data
> ต้องการเพียงข้อมูลที่เป็น เดือน/วัน/ปี จาก Column InvoiceDate

### แยก เดือน/วัน/ปี ออกจาก เวลา

In [ ]:
data['InvoiceDate'][0].split()[0]

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'12/01/2010'

In [ ]:
date = []
for i in range(len(data['InvoiceDate'])):
  x = data['InvoiceDate'][i].split()[0]
  date.append(x)
#set(L)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### แยก เดือน/วัน/ปี ออกจากกัน

In [ ]:
day =[]
month =[]
year =[]
for i in range(len(data['InvoiceDate'])):
  x = date[i].split('/')[0]
  y = date[i].split('/')[1]
  z = date[i].split('/')[2]
  month.append(x)
  day.append(y)
  year.append(z)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### นำ data ใหม่ไปใช้

In [ ]:
data_test_nonclean = data

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
data_test_nonclean['date'] = date
data_test_nonclean['day'] = day
data_test_nonclean['month'] = month
data_test_nonclean['year'] = year

# data_test_nonclean >>> ข้อมูลยังไม่อยู่ในมาตรฐานเดียวกัน

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
set(data_test_nonclean['date'])

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'01/04/2011',
 '01/05/2011',
 '01/06/2011',
 '01/07/2011',
 '01/09/2011',
 '01/10/2011',
 '01/11/2011',
 '01/12/2011',
 '02/01/2011',
 '02/02/2011',
 '02/03/2011',
 '02/04/2011',
 '02/06/2011',
 '02/07/2011',
 '02/08/2011',
 '02/09/2011',
 '02/10/2011',
 '02/11/2011',
 '03/01/2011',
 '03/02/2011',
 '03/03/2011',
 '03/04/2011',
 '03/06/2011',
 '03/07/2011',
 '03/08/2011',
 '03/09/2011',
 '03/10/2011',
 '03/11/2011',
 '04/01/2011',
 '04/03/2011',
 '04/04/2011',
 '04/05/2011',
 '04/06/2011',
 '04/07/2011',
 '04/08/2011',
 '04/10/2011',
 '04/11/2011',
 '04/12/2011',
 '05/01/2011',
 '05/03/2011',
 '05/04/2011',
 '05/05/2011',
 '05/06/2011',
 '05/08/2011',
 '05/09/2011',
 '05/10/2011',
 '05/11/2011',
 '05/12/2011',
 '06/01/2011',
 '06/02/2011',
 '06/03/2011',
 '06/05/2011',
 '06/06/2011',
 '06/07/2011',
 '06/08/2011',
 '06/09/2011',
 '06/10/2011',
 '06/12/2011',
 '07/01/2011',
 '07/03/2011',
 '07/04/2011',
 '07/05/2011',
 '07/06/2011',
 '07/07/2011',
 '07/08/2011',
 '07/10/2011',
 '07/11/20

In [ ]:
set(data_test_nonclean['day'])

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31'}

### แปลงวัน

In [ ]:
x=[]
for i in range(1,32):
  x.append(i)
print(x)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
data_test0 = data_test_nonclean.replace(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10','11',
                                         '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
                                         '23', '24', '25', '26', '27', '28', '29', '30', '31'],x)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-65-211ff669797f>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_test0 = data_test_nonclean.replace(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10','11',


In [ ]:
data_test_nonclean.head()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,day,month,year
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/01/2010 08:26,2.55,17850.0,United Kingdom,12/01/2010,01,12,2010
1,536365,71053,WHITE METAL LANTERN,6,12/01/2010 08:26,3.39,17850.0,United Kingdom,12/01/2010,01,12,2010
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/01/2010 08:26,2.75,17850.0,United Kingdom,12/01/2010,01,12,2010
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/01/2010 08:26,3.39,17850.0,United Kingdom,12/01/2010,01,12,2010
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/01/2010 08:26,3.39,17850.0,United Kingdom,12/01/2010,01,12,2010


In [ ]:
data_test0.head()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,day,month,year
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/01/2010 08:26,2.55,17850.0,United Kingdom,12/01/2010,1,12,2010
1,536365,71053,WHITE METAL LANTERN,6,12/01/2010 08:26,3.39,17850.0,United Kingdom,12/01/2010,1,12,2010
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/01/2010 08:26,2.75,17850.0,United Kingdom,12/01/2010,1,12,2010
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/01/2010 08:26,3.39,17850.0,United Kingdom,12/01/2010,1,12,2010
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/01/2010 08:26,3.39,17850.0,United Kingdom,12/01/2010,1,12,2010


### แปลงเดือน

In [ ]:
set(data_test0['month'])

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{1,
 '1',
 10,
 11,
 12,
 2,
 '2',
 3,
 '3',
 4,
 '4',
 5,
 '5',
 6,
 '6',
 7,
 '7',
 8,
 '8',
 9,
 '9'}

In [ ]:
data_test = data_test0.replace(to_replace=["1", "2",'3','4','5','6','7','8','9'],value=[1,2,3,4,5,6,7,8,9])

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-69-9808b4c818b5>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_test = data_test0.replace(to_replace=["1", "2",'3','4','5','6','7','8','9'],value=[1,2,3,4,5,6,7,8,9])


In [ ]:
data_test.isnull().any()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0
InvoiceNo,False
StockCode,False
Description,True
Quantity,False
InvoiceDate,False
UnitPrice,False
CustomerID,True
Country,False
date,False
day,False


In [ ]:
data_test_nonclean[data_test_nonclean['month']=='01']

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,day,month,year
42481,539993,22386,JUMBO BAG PINK POLKADOT,10,01/04/2011 10:00,1.95,13313.0,United Kingdom,01/04/2011,04,01,2011
42482,539993,21499,BLUE POLKADOT WRAP,25,01/04/2011 10:00,0.42,13313.0,United Kingdom,01/04/2011,04,01,2011
42483,539993,21498,RED RETROSPOT WRAP,25,01/04/2011 10:00,0.42,13313.0,United Kingdom,01/04/2011,04,01,2011
42484,539993,22379,RECYCLING BAG RETROSPOT,5,01/04/2011 10:00,2.10,13313.0,United Kingdom,01/04/2011,04,01,2011
42485,539993,20718,RED RETROSPOT SHOPPER BAG,10,01/04/2011 10:00,1.25,13313.0,United Kingdom,01/04/2011,04,01,2011
...,...,...,...,...,...,...,...,...,...,...,...,...
55385,540992,21912,VINTAGE SNAKES & LADDERS,2,01/12/2011 16:57,3.75,17364.0,United Kingdom,01/12/2011,12,01,2011
55386,540992,51008,AFGHAN SLIPPER SOCK PAIR,10,01/12/2011 16:57,3.45,17364.0,United Kingdom,01/12/2011,12,01,2011
55387,540992,21508,VINTAGE KID DOLLY CARD,12,01/12/2011 16:57,0.42,17364.0,United Kingdom,01/12/2011,12,01,2011
55388,540992,22716,CARD CIRCUS PARADE,12,01/12/2011 16:57,0.42,17364.0,United Kingdom,01/12/2011,12,01,2011


In [ ]:
data_test[data_test['month']==1]

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,day,month,year
42481,539993,22386,JUMBO BAG PINK POLKADOT,10,01/04/2011 10:00,1.95,13313.0,United Kingdom,01/04/2011,4,1,2011
42482,539993,21499,BLUE POLKADOT WRAP,25,01/04/2011 10:00,0.42,13313.0,United Kingdom,01/04/2011,4,1,2011
42483,539993,21498,RED RETROSPOT WRAP,25,01/04/2011 10:00,0.42,13313.0,United Kingdom,01/04/2011,4,1,2011
42484,539993,22379,RECYCLING BAG RETROSPOT,5,01/04/2011 10:00,2.10,13313.0,United Kingdom,01/04/2011,4,1,2011
42485,539993,20718,RED RETROSPOT SHOPPER BAG,10,01/04/2011 10:00,1.25,13313.0,United Kingdom,01/04/2011,4,1,2011
...,...,...,...,...,...,...,...,...,...,...,...,...
77623,542745,22138,BAKING SET 9 PIECE RETROSPOT,13,1/31/2011 16:47,4.95,15881.0,United Kingdom,1/31/2011,31,1,2011
77624,542746,21034,REX CASH+CARRY JUMBO SHOPPER,2,1/31/2011 16:48,0.95,15881.0,United Kingdom,1/31/2011,31,1,2011
77625,C542748,22969,HOMEMADE JAM SCENTED CANDLES,-12,1/31/2011 16:53,1.45,15881.0,United Kingdom,1/31/2011,31,1,2011
77626,C542753,22649,STRAWBERRY FAIRY CAKE TEAPOT,-1,1/31/2011 16:57,4.95,15881.0,United Kingdom,1/31/2011,31,1,2011


### เเบ่งทวีป

In [ ]:
set(data_test['Country'])

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'Australia',
 'Austria',
 'Bahrain',
 'Belgium',
 'Brazil',
 'Canada',
 'Channel Islands',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'EIRE',
 'European Community',
 'Finland',
 'France',
 'Germany',
 'Greece',
 'Hong Kong',
 'Iceland',
 'Israel',
 'Italy',
 'Japan',
 'Lebanon',
 'Lithuania',
 'Malta',
 'Netherlands',
 'Norway',
 'Poland',
 'Portugal',
 'RSA',
 'Saudi Arabia',
 'Singapore',
 'Spain',
 'Sweden',
 'Switzerland',
 'USA',
 'United Arab Emirates',
 'United Kingdom',
 'Unspecified'}

In [ ]:
# prompt: แบ่งงแต่ละประเทศ เป็นทวีปตามนี้ นี่คือรายการประเทศพร้อมทวีปที่แต่ละประเทศตั้งอยู่:
# ทวีปเอเชีย (Asia):
# 	•	Bahrain
# 	•	Cyprus (อยู่ในตะวันออกกลาง แต่อาจถูกจัดในยุโรปบางครั้ง)
# 	•	Hong Kong
# 	•	Israel
# 	•	Japan
# 	•	Lebanon
# 	•	Saudi Arabia
# 	•	Singapore
# 	•	United Arab Emirates
# ทวีปยุโรป (Europe):
# 	•	Austria
# 	•	Belgium
# 	•	Channel Islands (เขตปกครองพิเศษของสหราชอาณาจักร)
# 	•	Czech Republic
# 	•	Denmark
# 	•	EIRE (Ireland)
# 	•	European Community (ภูมิภาคในยุโรป)
# 	•	Finland
# 	•	France
# 	•	Germany
# 	•	Greece
# 	•	Iceland
# 	•	Italy
# 	•	Lithuania
# 	•	Malta
# 	•	Netherlands
# 	•	Norway
# 	•	Poland
# 	•	Portugal
# 	•	Spain
# 	•	Sweden
# 	•	Switzerland
# 	•	United Kingdom
# ทวีปอเมริกาเหนือ (North America):
# 	•	Canada
# 	•	USA (United States of America)
# ทวีปอเมริกาใต้ (South America):
# 	•	Brazil
# ทวีปออสเตรเลียและโอเชียเนีย (Oceania):
# 	•	Australia
# ทวีปแอฟริกา (Africa):
# 	•	RSA (Republic of South Africa)
# ไม่ระบุ (Unspecified):
# 	•	Unspecified
# หากต้องการข้อมูลเพิ่มเติมหรือแผนที่ประกอบบอกได้นะครับ!

def assign_continent(country):
    asia = ['Bahrain', 'Cyprus', 'Hong Kong', 'Israel', 'Japan', 'Lebanon',
            'Saudi Arabia', 'Singapore', 'United Arab Emirates']
    europe = ['Austria', 'Belgium', 'Channel Islands', 'Czech Republic', 'Denmark',
              'EIRE', 'European Community', 'Finland', 'France', 'Germany', 'Greece',
              'Iceland', 'Italy', 'Lithuania', 'Malta', 'Netherlands', 'Norway',
              'Poland', 'Portugal', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom']
    north_america = ['Canada', 'USA']
    south_america = ['Brazil']
    oceania = ['Australia']
    africa = ['RSA']

    if country in asia:
        return 'Asia'
    elif country in europe:
        return 'Europe'
    elif country in north_america:
        return 'North America'
    elif country in south_america:
        return 'South America'
    elif country in oceania:
        return 'Oceania'
    elif country in africa:
        return 'Africa'
    else:
        return 'Unspecified'

# Assuming 'data' is your DataFrame
data_test['Continent'] = data_test['Country'].apply(assign_continent)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
data_test

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,day,month,year,Continent
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/01/2010 08:26,2.55,17850.0,United Kingdom,12/01/2010,1,12,2010,Europe
1,536365,71053,WHITE METAL LANTERN,6,12/01/2010 08:26,3.39,17850.0,United Kingdom,12/01/2010,1,12,2010,Europe
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/01/2010 08:26,2.75,17850.0,United Kingdom,12/01/2010,1,12,2010,Europe
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/01/2010 08:26,3.39,17850.0,United Kingdom,12/01/2010,1,12,2010,Europe
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/01/2010 08:26,3.39,17850.0,United Kingdom,12/01/2010,1,12,2010,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/09/2011 12:50,0.85,12680.0,France,12/09/2011,9,12,2011,Europe
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/09/2011 12:50,2.10,12680.0,France,12/09/2011,9,12,2011,Europe
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/09/2011 12:50,4.15,12680.0,France,12/09/2011,9,12,2011,Europe
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/09/2011 12:50,4.15,12680.0,France,12/09/2011,9,12,2011,Europe


In [ ]:
#data_test = data_test.dropna(subset=['CustomerID'])

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import pandas as pd

from mlxtend.frequent_patterns import apriori, association_rules

# 3. Define Seasons

def assign_season (month):

  if month in [3, 4, 5]:

    return 'Spring'

  elif month in [6, 7, 8]:

    return 'Summer'

  elif month in [9, 10, 11]:

    return 'Autumn'

  else:

    return 'Winter'

# Add 'Season' column to data_test (Assuming this is already done)

data_test['InvoiceDate'] = pd.to_datetime(data_test['InvoiceDate']) # Convert to datetime

data_test ['Season'] = data_test['InvoiceDate'].dt.month.apply(assign_season)
data_test.head(10)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,day,month,year,Continent,Season
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,12/01/2010,1,12,2010,Europe,Winter
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,12/01/2010,1,12,2010,Europe,Winter
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,12/01/2010,1,12,2010,Europe,Winter
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,12/01/2010,1,12,2010,Europe,Winter
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,12/01/2010,1,12,2010,Europe,Winter
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom,12/01/2010,1,12,2010,Europe,Winter
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom,12/01/2010,1,12,2010,Europe,Winter
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom,12/01/2010,1,12,2010,Europe,Winter
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom,12/01/2010,1,12,2010,Europe,Winter
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom,12/01/2010,1,12,2010,Europe,Winter


In [ ]:
!pip install apyori
from apyori import apriori

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# prompt: หาสินค้าที่ถูกซื้อคู่กันใน winter โดยใช้ apriori algorithm

# Filter transactions for the winter season
winter_transactions = data_test[data_test['Season'] == 'Winter']

# Group by CustomerID and get the StockCodes as lists
winter_transactions = winter_transactions.groupby('CustomerID')['StockCode'].apply(list)

# Convert the list of lists to a format suitable for Apriori
winter_transactions_list = winter_transactions.values.tolist()
records = []
for i in range(len(winter_transactions_list)):
    records.append([str(winter_transactions_list[i][j]) for j in range(0, len(winter_transactions_list[i]))])

# Apply Apriori algorithm
association_rules = apriori(records, min_support=0.03, min_confidence=0.3, min_lift=3, min_length=2)
association_results = list(association_rules)


# Print the results or further process them as needed
for item in association_results:
    pair = item[0]
    items = [x for x in pair]
    print("Rule: " + items[0] + " -> " + items[1])
    print("Support: " + str(item[1]))
    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("=====================================")

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Rule: 20725 -> 20726
Support: 0.034183919114106884
Confidence: 0.3901098901098901
Lift: 6.866595269137642
Rule: 20725 -> 20727
Support: 0.03996148290804044
Confidence: 0.45604395604395603
Lift: 7.121829298521027
Rule: 20725 -> 20728
Support: 0.0365912373615792
Confidence: 0.41758241758241765
Lift: 7.167922986104805
Rule: 20725 -> 22382
Support: 0.03948001925854598
Confidence: 0.4505494505494506
Lift: 6.732310854613014
Rule: 20725 -> 22383
Support: 0.036109773712084736
Confidence: 0.41208791208791207
Lift: 6.686770260989011
Rule: 20725 -> 22384
Support: 0.036109773712084736
Confidence: 0.41208791208791207
Lift: 7.378505115574081
Rule: 85099B -> 20725
Support: 0.036109773712084736
Confidence: 0.41208791208791207
Lift: 3.872880513152006
Rule: 20726 -> 22382
Support: 0.030813673567645642
Confidence: 0.5423728813559322
Lift: 8.104377514937203
Rule: 20727 -> 22383
Support: 0.031295137217140105
Confidence: 0.48872180451127817
Lift: 7.930274906015037
Rule: 21166 -> 21175
Support: 0.03514684641

In [ ]:
set(data_test[(data_test['StockCode']=='85123A')] ['Description'])

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'?',
 'CREAM HANGING HEART T-LIGHT HOLDER',
 'WHITE HANGING HEART T-LIGHT HOLDER',
 'wrongly marked carton 22804'}

In [ ]:
set(data_test[(data_test['StockCode']=='22804')] ['Description'])

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'CANDLEHOLDER PINK HANGING HEART',
 'PINK HANGING HEART T-LIGHT HOLDER',
 'Wrongly mrked had 85123a in box'}

In [ ]:
# prompt: ทำเหมือนโค้ดบรรทัดที่ 178 แค่เปลี่ยนจาก winter เป็น spring

# Filter transactions for the spring season
spring_transactions = data_test[data_test['Season'] == 'Spring']

# Group by CustomerID and get the StockCodes as lists
spring_transactions = spring_transactions.groupby('CustomerID')['StockCode'].apply(list)

# Convert the list of lists to a format suitable for Apriori
spring_transactions_list = spring_transactions.values.tolist()
records = []
for i in range(len(spring_transactions_list)):
    records.append([str(spring_transactions_list[i][j]) for j in range(0, len(spring_transactions_list[i]))])

# Apply Apriori algorithm
association_rules = apriori(records, min_support=0.03, min_confidence=0.3, min_lift=3, min_length=2)
association_results = list(association_rules)


# Print the results or further process them as needed
for item in association_results:
    pair = item[0]
    items = [x for x in pair]
    print("Rule: " + items[0] + " -> " + items[1])
    print("Support: " + str(item[1]))
    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("=====================================")

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Rule: 20724 -> 20723
Support: 0.031080414405525406
Confidence: 0.6702127659574467
Lift: 11.512892174540207
Rule: 20725 -> 20724
Support: 0.030093734583127776
Confidence: 0.516949152542373
Lift: 4.421333047271688
Rule: 22355 -> 20724
Support: 0.030587074494326592
Confidence: 0.5254237288135594
Lift: 11.210883140053525
Rule: 20725 -> 20726
Support: 0.0493339911198816
Confidence: 0.4219409282700422
Lift: 5.55372897145049
Rule: 20725 -> 20727
Support: 0.05673408978786384
Confidence: 0.48523206751054854
Lift: 5.751844449379426
Rule: 20725 -> 20728
Support: 0.05525407005426739
Confidence: 0.47257383966244726
Lift: 5.411904932179551
Rule: 21931 -> 20725
Support: 0.03206709422792304
Confidence: 0.5078125
Lift: 4.343189609704641
Rule: 20725 -> 21936
Support: 0.041440552540700545
Confidence: 0.35443037974683544
Lift: 5.282576321667908
Rule: 20725 -> 22382
Support: 0.04982733103108041
Confidence: 0.4261603375527426
Lift: 4.964523012755225
Rule: 20725 -> 22383
Support: 0.05673408978786384
Confiden

In [ ]:
set(data[(data['StockCode']=='21086')] ['Description'])

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'SET/6 RED SPOTTY PAPER CUPS'}

In [ ]:
set(data[(data['StockCode']=='21094')] ['Description'])

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'SET/6 RED SPOTTY PAPER PLATES'}

In [ ]:
# prompt: ทำเหมือนโค้ดบรรทัดที่ 178 แค่เปลี่ยนจาก winter เป็น summer

# Filter transactions for the summer season
summer_transactions = data_test[data_test['Season'] == 'Summer']

# Group by CustomerID and get the StockCodes as lists
summer_transactions = summer_transactions.groupby('CustomerID')['StockCode'].apply(list)

# Convert the list of lists to a format suitable for Apriori
summer_transactions_list = summer_transactions.values.tolist()
records = []
for i in range(len(summer_transactions_list)):
    records.append([str(summer_transactions_list[i][j]) for j in range(0, len(summer_transactions_list[i]))])

# Apply Apriori algorithm
association_rules = apriori(records, min_support=0.03, min_confidence=0.3, min_lift=1, min_length=2)
association_results = list(association_rules)


# Print the results or further process them as needed
for item in association_results:
    pair = item[0]
    items = [x for x in pair]
    print("Rule: " + items[0] + " -> " + items[1])
    print("Support: " + str(item[1]))
    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("=====================================")

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Rule: 22386 -> 20712
Support: 0.030607124937280482
Confidence: 0.61
Lift: 7.109532163742689
Rule: 85099B -> 20712
Support: 0.03562468640240843
Confidence: 0.7100000000000001
Lift: 5.299737827715356
Rule: 20719 -> 20723
Support: 0.030607124937280482
Confidence: 0.5495495495495495
Lift: 9.60747589694958
Rule: 20719 -> 20724
Support: 0.03311590566984446
Confidence: 0.5945945945945946
Lift: 8.843485276321097
Rule: 20719 -> 23204
Support: 0.030607124937280482
Confidence: 0.5495495495495495
Lift: 9.281798747900442
Rule: 20724 -> 20723
Support: 0.03462117410938284
Confidence: 0.6052631578947368
Lift: 9.002160251374704
Rule: 23204 -> 20723
Support: 0.03261414952333166
Confidence: 0.5701754385964912
Lift: 9.63016651798989
Rule: 20725 -> 20724
Support: 0.03361766181635725
Confidence: 0.5
Lift: 3.788973384030418
Rule: 22355 -> 20724
Support: 0.03361766181635725
Confidence: 0.5
Lift: 9.142201834862385
Rule: 22356 -> 20724
Support: 0.03261414952333166
Confidence: 0.48507462686567154
Lift: 10.508192

In [ ]:
set(data[(data['StockCode']=='23285')] ['Description'])

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'PINK VINTAGE SPOT BEAKER'}

In [ ]:
set(data[(data['StockCode']=='23286')] ['Description'])

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'BLUE VINTAGE SPOT BEAKER'}

In [ ]:
# prompt: ทำเหมือนโค้ดบรรทัดที่ 178 แค่เปลี่ยนจาก winter เป็น autumn

# Filter transactions for the autumn season
autumn_transactions = data_test[data_test['Season'] == 'Autumn']

# Group by CustomerID and get the StockCodes as lists
autumn_transactions = autumn_transactions.groupby('CustomerID')['StockCode'].apply(list)

# Convert the list of lists to a format suitable for Apriori
autumn_transactions_list = autumn_transactions.values.tolist()
records = []
for i in range(len(autumn_transactions_list)):
    records.append([str(autumn_transactions_list[i][j]) for j in range(0, len(autumn_transactions_list[i]))])

# Apply Apriori algorithm
association_rules = apriori(records, min_support=0.03, min_confidence=0.3, min_lift=3, min_length=2)
association_results = list(association_rules)


# Print the results or further process them as needed
for item in association_results:
    pair = item[0]
    items = [x for x in pair]
    print("Rule: " + items[0] + " -> " + items[1])
    print("Support: " + str(item[1]))
    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("=====================================")

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Rule: 85099B -> 20724
Support: 0.034122562674094706
Confidence: 0.5326086956521738
Lift: 3.98346920289855
Rule: 20725 -> 20726
Support: 0.03725626740947075
Confidence: 0.4163424124513619
Lift: 6.793951185001769
Rule: 20725 -> 20727
Support: 0.04805013927576602
Confidence: 0.5369649805447472
Lift: 6.320341902149647
Rule: 20725 -> 20728
Support: 0.03934540389972145
Confidence: 0.43968871595330744
Lift: 5.900869122513547
Rule: 20725 -> 22382
Support: 0.045612813370473536
Confidence: 0.5097276264591439
Lift: 5.740932326237887
Rule: 20725 -> 22383
Support: 0.044916434540389974
Confidence: 0.5019455252918288
Lift: 6.493637606478074
Rule: 20725 -> 22384
Support: 0.044916434540389974
Confidence: 0.5019455252918288
Lift: 7.392756659682729
Rule: 20725 -> 23202
Support: 0.03064066852367688
Confidence: 0.3424124513618677
Lift: 4.063671736823488
Rule: 23203 -> 20725
Support: 0.03586350974930362
Confidence: 0.40077821011673154
Lift: 3.519984768976309
Rule: 20725 -> 23206
Support: 0.04387186629526462

In [ ]:
set(data[(data['StockCode']=='22699')] ['Description'])

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'ROSES REGENCY TEACUP AND SAUCER '}

In [ ]:
set(data[(data['StockCode']=='22698')] ['Description'])

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'PINK REGENCY TEACUP AND SAUCER', nan}